In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
import re

In [ ]:
!pip install bert-extractive-summarizer
from summarizer import Summarizer

In [ ]:
def remove_media(text):
  # Remove link
  text = re.sub(r'\S*https?:\/\/\S*|\S*\.com\S*|\S*\.co\S*', '', text, flags=re.IGNORECASE)
  # Remove multiple spaces and replace with a single space
  text = re.sub("\s\s+" , " ", text)
  # Replace hyphens with periods to treat them as sentence boundaries
  text = re.sub(r'\s+\-', ".", text)
  # Remove spaces before a period
  text = re.sub(r'\s+\.', '.', text)
  # Remove double dot
  text = re.sub(r'\.{2,}', '.', text)
  # Remove unstandart char
  text = re.sub(r'[^\x00-\x7F]+', '', text)
  # Ensure that there is a space after every period (dot)
  text = re.sub(r'\.(\S)', r'. \1', text)
  # List of cities or countries to remove (add more names as needed)
  cities_countries = ["yogya", "jawa", "jakarta", "solo", "dki", "cnn", "indonesia", "cnn indonesia", "kompas", "tempo", "detik", "cnbc", "liputan 6", "merdeka", "tribunnews", "suara", "media", "republika", "medcom", "viva", "pos", "jppn"]
  # Create a regex pattern to match any of the city or country names at the start
  pattern = r'^(' + '|'.join(cities_countries) + r')\b[\s,:-]*'
  # Remove city or country names at the beginning of the text
  text = re.sub(pattern, '', text, flags=re.IGNORECASE)
  text = re.sub(pattern, '', text, flags=re.IGNORECASE)
  summary = Summarizer('distilbert-base-uncased')
  result = summary(text, num_sentences=1)
  return result


In [ ]:
df = pd.read_csv('/content/Political_Data.csv', sep=',', encoding='utf-8')
df.head(10)

,Deskripsi,Jenis_Konflik,Mediasi
0,TEMPO.CO YOGYA -Ketua Umum Partai Demokrat Ana...,Konflik Kepengurusan Partai,Musyawarah
1,TEMPO.CO Jakarta - Sudah beberapa pekan Partai...,Konflik Kepengurusan Partai,Musyawarah
2,JAKARTA KOMPAS.com — Mantan Ketua Umum DPP Par...,Konflik Kepengurusan Partai,Musyawarah
3,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...,Konflik Kepengurusan Partai,Musyawarah
4,JAKARTA KOMPAS.com - Deputi Badan Pemenangan P...,Konflik Kepengurusan Partai,Musyawarah
5,JAKARTA KOMPAS.com - Presidium Koalisi Aksi Me...,Konflik Kepengurusan Partai,Musyawarah
6,JAKARTA KOMPAS.com - Mantan Wakil Ketua Dewan ...,Konflik Kepengurusan Partai,Musyawarah
7,JAKARTA KOMPAS.com- Kader yang telah dipecat D...,Konflik Kepengurusan Partai,Musyawarah
8,JAKARTA KOMPAS.com - Kepala Badan Komunikasi S...,Konflik Kepengurusan Partai,Musyawarah
9,JAKARTA KOMPAS.com - Kepala Badan Komunikasi S...,Konflik Kepengurusan Partai,Musyawarah


In [ ]:
sum_narrative = []
narrative = df.pop('Deskripsi')
for i in range(len(narrative)):
  text = narrative[i]
  result = remove_media(text)
  sum_narrative.append(result)

0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
for i in range(10):
  print(sum_narrative[i])

Ketua Umum Partai Demokrat Anas Urbaningrum meminta kader Demokrat dari pusat sampai daerah waspada dengan beredarnya isu dan tanda-tanda gerakan yang berencana mengganggu dan menggulingkan pemerintahan Presiden Susilo Bambang Yudhoyono.
Sudah beberapa pekan Partai Demokrat diguncang isu pelengseran Ketua Umum Anas Urbaningrum.
Mantan Ketua Umum DPP Partai Demokrat Anas Urbaningrum ketika menjalankan tugasnya mengaku pernah berseberangan dengan Ketua Dewan Pembina Susilo Bambang Yudhoyono terkait beberapa keputusan politik.
Mantan Ketua Umum DPP Partai Demokrat Anas Urbaningrum ketika menjalankan tugasnya mengaku pernah berseberangan dengan Ketua Dewan Pembina Susilo Bambang Yudhoyono terkait beberapa keputusan politik.
Deputi Badan Pemenangan Pemilu (Bappilu) DPP Kamhar Lakumani menegaskan hingga saat ini tidak ada Kongres Luar Biasa di Partai Demokrat.
Presidium Koalisi Aksi Menyelamatkan Indonesia (KAMI) mengaku pernah didatangi seseorang untuk terlibat dalam upaya atau penggulingan

In [ ]:
sum_narrative = np.array(sum_narrative)
df['Final_Deskripsi_Summary'] = sum_narrative
df.head()

,Jenis_Konflik,Mediasi,Final_Deskripsi_Summary
0,Konflik Kepengurusan Partai,Musyawarah,Ketua Umum Partai Demokrat Anas Urbaningrum me...
1,Konflik Kepengurusan Partai,Musyawarah,Sudah beberapa pekan Partai Demokrat diguncang...
2,Konflik Kepengurusan Partai,Musyawarah,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
3,Konflik Kepengurusan Partai,Musyawarah,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
4,Konflik Kepengurusan Partai,Musyawarah,Deputi Badan Pemenangan Pemilu (Bappilu) DPP K...


In [ ]:
df.to_csv('Political_Data_1.csv', index=False)